In [ ]:
import pandas as pd
import numpy as np
import re
import sys

from functools import reduce
from datetime import datetime

sys.path.append('../src')
import helpers

%load_ext autoreload
%autoreload

In [ ]:
%%time
absentee_data_url = 'https://www.maine.gov/sos/cec/elec/data/absentee-voter-file.txt'

absentee_data_raw = pd.read_csv(absentee_data_url, sep='|', skipfooter=5, engine='python')

In [ ]:
absentee_data = absentee_data_raw.rename(columns={
    "CG": "CONGRESS DIST",
    "SS": "STATE SENATE",
    "P": "PARTY",

})

absentee_data['REQDATE'] = absentee_data['REQDATE'].str.replace('/0202', '/2020')
absentee_data['REQDATE'] = pd.to_datetime(absentee_data['REQDATE'], format='%m/%d/%Y')

In [ ]:
group_data = absentee_data.set_index('REQDATE')
group_data['TOT'] = 1
group_data['PARTY-D'] = group_data['PARTY'] == 'D'
group_data['PARTY-R'] = group_data['PARTY'] == 'R'
group_data['PARTY-G'] = group_data['PARTY'] == 'G'
group_data['PARTY-U'] = group_data['PARTY'] == 'U'
group_data['CD-1'] = group_data['CONGRESS DIST'] == 1
group_data['CD-2'] = group_data['CONGRESS DIST'] == 2
group_data['REQTYPE-VOTER/FAMILY'] = group_data['REQTYPE'] == 'WV'
group_data['REQTYPE-WRITTEN 3P'] = group_data['REQTYPE'] == 'WP'
group_data['REQTYPE-ELECTRONIC'] = group_data['REQTYPE'] == 'ER'
group_data['REQTYPE-IN PERSON'] = group_data['REQTYPE'] == 'VP'
group_data['REQTYPE-UOCAVA'] = group_data['REQTYPE'] == 'UR'
group_data['REQTYPE-PHONE'] = group_data['REQTYPE'] == 'TR'

group_columns = ['TOT',
                 'PARTY-D', 'PARTY-R', 'PARTY-G', 'PARTY-U',
                 'CD-1', 'CD-2',
                 'REQTYPE-VOTER/FAMILY', 'REQTYPE-WRITTEN 3P', 'REQTYPE-ELECTRONIC', 'REQTYPE-IN PERSON', 'REQTYPE-UOCAVA', 'REQTYPE-PHONE']
data_by_week = group_data[group_columns].resample('W-SAT').sum()

recent_data_by_week = data_by_week[datetime(2020, 3, 4):].copy()
recent_data_by_week.loc['EARLIER'] = data_by_week[:datetime(2020, 3, 4)].sum()
recent_data_by_week = recent_data_by_week.loc[['EARLIER', *recent_data_by_week.index[0:-1]]]
recent_data_by_week.loc['TOT'] = recent_data_by_week.sum()
recent_data_by_week['AGG-TOT'] = recent_data_by_week['TOT'].cumsum()
recent_data_by_week.loc['TOT', 'AGG-TOT'] = 0
recent_data_by_week = recent_data_by_week.astype(int)

In [ ]:
recent_data_by_week.to_excel('./out/2020 Primary Weekly Analysis.xlsx')